In [ ]:
total_samples = 20000

In [ ]:
f = open("/content/drive/MyDrive/small_train_"+str(total_samples)+".txt", "w")
for line in train.take(total_samples):
  f.write(line.numpy().decode()+'\n')
f.close()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import numpy as np
import os
import re
import string
import random
import tensorflow_datasets as tfds
import pickle
from keras.layers.normalization import BatchNormalization

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [4]:
train = tf.data.TextLineDataset("/content/drive/MyDrive/data/sentences_file.txt")

In [5]:
def custom_standard(input_string):
    return tf.strings.lower(input_string)

In [6]:
maxlen=20 # max sequence length
vectorize_layer = TextVectorization(
    standardize=custom_standard,
    max_tokens=None,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

In [7]:
vectorize_layer.adapt(train)

In [8]:
for i in train.take(2):
  print(i.numpy().decode())
# data is lowercased

ye to hona hi tha .... kabhi kisi neta ko saja mili hi ... always clean chit .. 
hey frd . hw r u ? msg now 


In [9]:
vocab = vectorize_layer.get_vocabulary()
vocab_size = len(vocab)

In [10]:
print(vocab_size)

28376


In [11]:
def prepare_dataset(text):
    '''
    text = [1,2,3,4,5] seq_len = 4
    x = [1,2,3,4] y = [2,3,4,5]
    '''
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

In [12]:
train_ds = train.map(prepare_dataset) # train_ds[0] = tuple(x[0],y[0])

In [ ]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.MapDataset

In [ ]:
# data is tokenised and ready for the model
for i in train_ds.take(1):
  print(i[0].numpy())
  print(i[1].numpy())

[[  18    4  193   19   48   28  216   87  221    5  780 1054   19   13
   922 1641 3479   16    0    0]]
[[   4  193   19   48   28  216   87  221    5  780 1054   19   13  922
  1641 3479   16    0    0    0]]


In [13]:
seq_length = 20
embed_dim = 50
lstm_cells = 50
dense_cells = 100

In [18]:
def complex_naive():
  model = Sequential()
  model.add(Embedding(vocab_size,embed_dim, input_length=seq_length))
  model.add(LSTM(lstm_cells, return_sequences=True))
  model.add(Dense(dense_cells, activation='relu'))
  model.add( tf.keras.layers.Dropout(0.2))
  model.add(Dense(vocab_size))
  #loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile("adam", loss='sparse_categorical_crossentropy')
  return model

In [ ]:
def simple_naive():
  model = Sequential()
  model.add(Embedding(vocab_size,embed_dim, input_length=seq_length))
  model.add(LSTM(lstm_cells, return_sequences=True))
  model.add(Dense(vocab_size))
  #loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile("adam",loss='sparse_categorical_crossentropy')
  return model

In [20]:
#simple naive
class Custom_Saver_for_complex_naive(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(epoch%3 == 0):
        self.model.save("/content/drive/MyDrive/naive_complex_model/"+str(epoch)+"/model/")
        #self.model.save("/content/drive/MyDrive/naive_model/naive_simple_model_"+str(epoch)+".hd5")

In [19]:
naive_complex_model = complex_naive()
print(naive_complex_model.summary())
saver_complex = Custom_Saver_for_complex_naive()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            1418800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 50)            20200     
_________________________________________________________________
dense (Dense)                (None, 20, 100)           5100      
_________________________________________________________________
dropout (Dropout)            (None, 20, 100)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 20, 28376)         2865976   
Total params: 4,310,076
Trainable params: 4,310,076
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print(type(train_ds))

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [22]:
naive_complex_model.fit(train_ds,epochs=10,callbacks=[saver_complex])

Epoch 1/10
16220/16220 [==============================] - 286s 16ms/step - loss: 10.2401


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/0/model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/0/model/assets


Epoch 2/10
16220/16220 [==============================] - 265s 16ms/step - loss: 9.6619
Epoch 3/10
16220/16220 [==============================] - 260s 16ms/step - loss: 7.6091
Epoch 4/10
16220/16220 [==============================] - 259s 16ms/step - loss: 7.6078
Epoch 5/10
16220/16220 [==============================] - 256s 16ms/step - loss: 7.6078


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/4/model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/4/model/assets


Epoch 6/10
16220/16220 [==============================] - 254s 16ms/step - loss: 7.6078
Epoch 7/10
16220/16220 [==============================] - 270s 17ms/step - loss: 7.6077
Epoch 8/10
16220/16220 [==============================] - 262s 16ms/step - loss: 7.6078
Epoch 9/10
16220/16220 [==============================] - 272s 17ms/step - loss: 7.6077


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/8/model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/8/model/assets


Epoch 10/10
16220/16220 [==============================] - 288s 18ms/step - loss: 7.6078


In [24]:
naive_complex_model.save("/content/drive/MyDrive/naive_complex_model/10/model/")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/10/model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/naive_complex_model/10/model/assets


In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/naive_model/naive_simple_model_20')

In [ ]:
class TextGenerator():
    def __init__(
        self, max_tokens, start_tokens, index_to_word, top_k=10,maxlen=20):
        self.max_tokens = max_tokens
        self.start_tokens = start_tokens
        self.index_to_word = index_to_word
        self.k = top_k
        self.maxlen=maxlen

    def sample_from(self, logits):
        logits, indices = tf.math.top_k(logits, k=self.k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def detokenize(self, number):
        return self.index_to_word[number]

    def get_sen(self, model):
        start_tokens = [_ for _ in self.start_tokens]
        num_tokens_generated = 0
        tokens_generated = []
        while num_tokens_generated <= self.max_tokens:
            pad_len = self.maxlen - len(start_tokens)
            sample_index = len(start_tokens) - 1
            if pad_len < 0:
                x = start_tokens[:self.maxlen]
                sample_index = self.maxlen - 1
            elif pad_len > 0:
                x = start_tokens + [0] * pad_len
            else:
                x = start_tokens
            x = np.array([x])
            y = model.predict(x)
            sample_token = self.sample_from(y[0][sample_index])
            tokens_generated.append(sample_token)
            start_tokens.append(sample_token)
            num_tokens_generated = len(tokens_generated)
        txt = " ".join(
            [self.detokenize(_) for _ in self.start_tokens + tokens_generated]
        )
        return txt

In [ ]:
word_to_index = {}
for index, word in enumerate(vocab):
    word_to_index[word] = index
num_tokens_generated = 100
start_tokens = [word_to_index.get(_, 1) for _ in start_prompt.split()]

In [ ]:
start_prompt = "ye sab kya"
gen = TextGenerator(num_tokens_generated, start_tokens, vocab)

In [ ]:
gen.get_sen(model)

'ye sab kya waar pellet di prayer prayer sochana prayer potli tmari fenkte fenkte ilzaam fenkte smone krantikari prayer krantikari tmari potli potli govment fenkte patidar krantikari govment potli tmari prayer fenkte tmari waar patidar krantikari tmari prayer tmari krantikari tmari krantikari prayer potli potli tmari potli ilzaam patidar prayer fenkte krantikari ilzaam potli fenkte krantikari tmari prayer prayer potli krantikari krantikari patidar smone ilzaam patidar tmari govment smone fenkte potli potli smone tmari fenkte ilzaam waar patidar patidar ilzaam prayer prayer prayer smone govment tmari krantikari krantikari patidar potli potli ilzaam krantikari prayer smone fenkte tmari govment fenkte waar govment smone govment fenkte'